In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
import random
import lightgbm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
plt.close("all")
# random.seed(2021)


train_hour_df=pd.read_csv('./data/train_v2.csv')
task1_sota = pd.read_csv('submit/task1.txt')
task1_sota['amount'] = task1_sota['amount']/1e3
test_hour_df=pd.read_csv('./data/test_v2_periods.csv')#按0.5h计算
test_day=pd.read_csv('./data/test_v2_day.csv')#按天计算
wkd_df=pd.read_csv('./data/wkd_v1.csv')
wkd_df=wkd_df.rename(columns={'ORIG_DT':'date'})
train_df=train_hour_df.merge(wkd_df)
print(train_df.head(5))
#处理特征
def get_frt(df):
    df['WKD_TYP_CD']=df['WKD_TYP_CD'].map({'WN':0,'SN': 1, 'NH': 1, 'SS': 1, 'WS': 0})
    df['date']=pd.to_datetime(df['date'])
    df['dayofweek']=df['date'].dt.dayofweek+1
    df['day']=df['date'].dt.day
    df['month']=df['date'].dt.month
    df['year']=df['date'].dt.year
    df.drop(['date','post_id'],axis=1,inplace=True)
    return df
#为聚类做数据准备
def get_kmeans_train(df,Type=1):
    idx = list(df['date'].unique())
    if Type==1:
        df = df[df.post_id=='A'].groupby(['date','periods'])['amount'].sum()
    else:
        df = df[df.post_id=='B'].groupby(['date','periods'])['amount'].sum()
    ret = pd.DataFrame(columns=list(range(1,49)),index=idx)
    for item in idx:
        ret.loc[item] = df.loc[item].values.T
    ret.drop(list(range(1,18))+list(range(38,49)),axis=1,inplace=True)
    ret['date'] = pd.to_datetime(ret.index)
    ret['month'] = ret['date'].dt.month
    ret = ret[(ret.month>10)&(ret.month<=12)]
    # ret = ret[(ret.month == 10)]
    ret = ret.drop(['date','month'],axis=1)
    return ret
#train for lgb
tmp=train_df[['date','post_id','amount']].groupby(['date','post_id'],sort=False).agg('sum')
train_day_df=pd.DataFrame(tmp).reset_index()
train_day_df_A=train_day_df[train_day_df['post_id']=='A'].reset_index(drop=True)
train_day_df_B=train_day_df[train_day_df['post_id']=='B'].reset_index(drop=True)
train_day_df_A=train_day_df_A.merge(wkd_df)
train_day_df_B=train_day_df_B.merge(wkd_df)
train_day_df_A=get_frt(train_day_df_A)
train_day_df_A = train_day_df_A[(train_day_df_A.month>10)&(train_day_df_A.month<=12)]
# train_day_df_A = train_day_df_A[(train_day_df_A.month==10)]
train_day_df_B=get_frt(train_day_df_B)
train_day_df_B = train_day_df_B[(train_day_df_B.month>10)&(train_day_df_B.month<=12)]
# train_day_df_B = train_day_df_B[(train_day_df_B.month==10)]
train_day_df_A['amount']=train_day_df_A['amount']/1e3
train_day_df_B['amount']=train_day_df_B['amount']/1e3
#train for kmeans
kmeans_df_A = get_kmeans_train(train_df,1)#.reset_index()
kmeans_df_B = get_kmeans_train(train_df,2)#.reset_index()
#test
tmp=test_hour_df[['date','post_id']].groupby(['date','post_id'],sort=False).agg('sum')
test_day_df=pd.DataFrame(tmp).reset_index()
test_day_df_A=test_day_df[test_day_df['post_id']=='A'].reset_index(drop=True)
test_day_df_B=test_day_df[test_day_df['post_id']=='B'].reset_index(drop=True)
test_day_df_A=test_day_df_A.merge(wkd_df)
test_day_df_B=test_day_df_B.merge(wkd_df)
test_day_df_A=get_frt(test_day_df_A)
test_day_df_B=get_frt(test_day_df_B)
#训练集和测试集
print(train_day_df_A.shape,test_day_df_A.shape) #(1035, 6) (30, 5)
def lgb_cv(train_x, train_y, test_x,k):
    predictors = list(train_x.columns)
    train_x = train_x.values
    test_x = test_x.values
    folds = 10
    # seed = 2021
    # kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    kf = KFold(n_splits=folds, shuffle=True)
    train = np.zeros((train_x.shape[0]))
    test = np.zeros((test_x.shape[0]))
    test_pre = np.zeros((folds, test_x.shape[0]))
    test_pre_all = np.zeros((folds, test_x.shape[0]))
    cv_scores = []
    tpr_scores = []
    cv_rounds = []

    for i, (train_index, test_index) in enumerate(kf.split(train_x, train_y)):
        tr_x = train_x[train_index]
        tr_y = train_y[train_index]
        te_x = train_x[test_index]
        te_y = train_y[test_index]
        train_matrix = lightgbm.Dataset(tr_x, label=tr_y)
        test_matrix = lightgbm.Dataset(te_x, label=te_y)
        params = {
            'boosting_type': 'gbdt',
            'objective': 'multiclass',
            'num_class':k,
            'metrics':'multi_error',
            'num_leaves': 2 ** 5-1,
            'feature_fraction': 0.8,
            'bagging_fraction': 0.8,
            'learning_rate': 0.05,
            # 'seed': 2021,
            'nthread': 8,
            'verbose': -1,
        }
        num_round = 4000
        early_stopping_rounds = 100
        if test_matrix:
            model = lightgbm.train(params, train_matrix, num_round, valid_sets=test_matrix, verbose_eval=200,
                              #feval=tpr_eval_score,
                              early_stopping_rounds=early_stopping_rounds
                              )
            print("\n".join(("%s: %.2f" % x) for x in list(sorted(zip(predictors, model.feature_importance("gain")),
                        key=lambda x: x[1],reverse=True))[:10]))
            importance_list=[ x[0] for x in list(sorted(zip(predictors, model.feature_importance("gain")),
                        key=lambda x: x[1],reverse=True))]
            #print(importance_list)
            pre_prob = model.predict(te_x, num_iteration=model.best_iteration)#
            pred_prob = model.predict(test_x, num_iteration=model.best_iteration)#
            pre = [list(x).index(max(x)) for x in pre_prob]
            pred = [list(x).index(max(x)) for x in pred_prob]
            train[test_index] = pre
            test_pre[i, :] = pred
            cv_scores.append(accuracy_score(te_y, pre))
            cv_rounds.append(model.best_iteration)
            test_pre_all[i, :] = pred
        #
        print("cv_score is:", cv_scores)
    use_mean=True
    if use_mean:
        test[:] = test_pre.mean(axis=0)
    else:
        pass
    #
    print("val_mean:" , np.mean(cv_scores))
    print("val_std:", np.std(cv_scores))
    return train, test, test_pre_all, np.mean(cv_scores),importance_list



def predict(train_day_df_A,test_day_df_A,task1_sota,k1,kmeans_df_A,train_day_df_B,test_day_df_B,k2,kmeans_df_B,test_hour_df):
    select_frts = ['WKD_TYP_CD', 'year', 'month', 'day', 'dayofweek', 'amount']
    train_df = train_day_df_A  # 训练集A
    #     train_df=train_df[(train_df['year']==2020) & (train_df['month']>4)].reset_index(drop=True)
    test_df = test_day_df_A  # 测试集A
    test_df['amount'] = task1_sota[task1_sota.post_id == 'A']['amount'].values
    train_x = train_df[select_frts].copy()
    kmeans_df = kmeans_df_A
    kmeans = KMeans(n_clusters=k1)
    kmeans.fit(kmeans_df.values)
    groups = kmeans.predict(kmeans_df.values)
    centers = kmeans.cluster_centers_
    # import pdb
    # pdb.set_trace()
    train_y = np.array(groups)
    test_x = test_df[select_frts].copy()
    print(train_x.shape, train_y.shape, test_x.shape)
    lgb_train, lgb_test_A, sb, cv_scores, importance_list = lgb_cv(train_x, train_y, test_x, k1)
    centers_A = [[it * 1.0 / max(1, sum(item)) for it in item] for item in centers]
    train_df['label'] = train_y
    print(train_df['label'].value_counts())
    # print(lgb_test_A)
    #
    print("**" * 40)
    print("**" * 40)

    train_df = train_day_df_B  # 训练集A
    #     train_df=train_df[(train_df['year']==2020) & (train_df['month']>4)].reset_index(drop=True)
    test_df = test_day_df_B  # 测试集A
    test_df['amount'] = task1_sota[task1_sota.post_id == 'B']['amount'].values
    train_x = train_df[select_frts].copy()
    kmeans_df = kmeans_df_B
    kmeans = KMeans(n_clusters=k2)
    kmeans.fit(kmeans_df.values)
    groups = kmeans.predict(kmeans_df.values)
    centers = kmeans.cluster_centers_
    # groups, centers = KMeans(k2, kmeans_df.values, max_iters=50)
    train_y = np.array(groups)
    test_x = test_df[select_frts].copy()
    print(train_x.shape, train_y.shape, test_x.shape)
    lgb_train, lgb_test_B, sb, cv_scores, importance_list = lgb_cv(train_x, train_y, test_x, k2)
    centers_B = [[it * 1.0 / max(1, sum(item)) for it in item] for item in centers]
    print([sum(item) for item in centers_B])
    train_df['label'] = train_y
    print(train_df['label'].value_counts())

    dic = {}  # 记录A的每天各时段的工作量
    for i, item in enumerate(task1_sota[task1_sota.post_id == 'A'].values):
        date, pi, amount = item[0], item[1], item[2]
        dic[date + pi] = [amount * centers_A[int(lgb_test_A[i])][j] for j in range(20)]
    for i in range(len(test_hour_df)):
        if test_hour_df.loc[i, 'post_id'] == 'B':
            continue
        date = test_hour_df.loc[i, 'date']
        periods = test_hour_df.loc[i, 'periods']
        if periods < 38 and periods > 17:
            test_hour_df.loc[i, 'amount'] = dic[date + 'A'][periods - 18]

    dic = {}  # 记录B的每天各时段的工作量
    for i, item in enumerate(task1_sota[task1_sota.post_id == 'B'].values):
        date, pi, amount = item[0], item[1], item[2]
        dic[date + pi] = [amount * centers_B[int(lgb_test_B[i])][j] for j in range(20)]
    for i in range(len(test_hour_df)):
        if test_hour_df.loc[i, 'post_id'] == 'A':
            continue
        date = test_hour_df.loc[i, 'date']
        periods = test_hour_df.loc[i, 'periods']
        if periods < 38 and periods > 17:
            test_hour_df.loc[i, 'amount'] = dic[date + 'B'][periods - 18]
    test_hour_df = test_hour_df.fillna(0)
    return test_hour_df
    # test_hour_df['amount'] = test_hour_df['amount'].apply(lambda x: int(x * 1e3))
    # test_hour_df.to_csv('data/KMEANS_LGB/task2_1.txt', index=False, encoding='utf-8', sep=',')

if __name__=="__main__":
    summay = pd.DataFrame(index=range(len(test_hour_df['date'])))
    test_hour_df_ = test_hour_df.copy()
    k1 = 4
    k2 = 3
    for i in range(15):
        ret = predict(train_day_df_A,test_day_df_A,task1_sota,k1,kmeans_df_A,train_day_df_B,test_day_df_B,k2,kmeans_df_B,test_hour_df_)
        summay['amount'+str(i)] = ret['amount']
    summay['mean'] = summay.mean(axis=1)
    test_hour_df['amount'] =  summay['mean']
    test_hour_df['amount'] = test_hour_df['amount'].apply(lambda x:int(x * 1e3))
#     test_hour_df.to_csv('submit/task2_1.txt',index=False, encoding='utf-8', sep=',')